In [5]:
from PIL import Image
import numpy as np
import pandas as pd

def get_image_info(image):
    width, height = image.size
    resolution = f"Resolution: {width} x {height} = {width * height} pixels"
    return resolution

def count_colors_fast(image):
    pixels = np.array(image)
    h, w, c = pixels.shape
    pixels2 = pixels.reshape(h * w, c)
    view = pixels2.view(np.dtype((np.void, pixels2.dtype.itemsize * c)))
    unique, counts = np.unique(view, return_counts=True)
    unique_colors = list(map(tuple, unique.view(pixels2.dtype).reshape(-1, c)))
    return dict(zip(unique_colors, counts))

def analyze_image(image_path):   
    image = Image.open(image_path).convert("RGBA")
    resolution = get_image_info(image)
    color_counts = count_colors_fast(image)
    color_data = [(value, count) for value, count in color_counts.items()]
    df = pd.DataFrame(color_data, columns=["Value", "Count"]).sort_values(by="Count", ascending=False)
    print(resolution)
    return df, color_counts

def color_background(val):
    if isinstance(val, tuple):
        if len(val) == 4:
            r, g, b, a = val
            if a < 255:
                return f"background-color: rgba({r}, {g}, {b}, {a/255:.2f})"
            return f"background-color: rgb({r}, {g}, {b})"
        elif len(val) == 3:
            r, g, b = val
            return f"background-color: rgb({r}, {g}, {b})"
    return ""

def colorize(color_counts):
    color_data = [(value, value, count) for value, count in color_counts.items()]
    df = pd.DataFrame(color_data, columns=["Color", "Value", "Count"]).sort_values(by="Count", ascending=False)
    styled_df = df.style.map(color_background, subset=["Color"])
    styled_df = styled_df.format({"Color": ""})
    return styled_df

In [ ]:
image_path = "Lighthouse.jpg"  #<------Change this to the path of the image you want to analyze.
df, color_counts = analyze_image(image_path)
df

Resolution: 1024 x 768 = 786432 pixels


,Value,Count
52,"(1, 1, 1, 255)",5347
95,"(2, 2, 2, 255)",4876
0,"(0, 0, 0, 255)",4761
96,"(2, 2, 4, 255)",4301
151,"(3, 3, 5, 255)",4190
...,...,...
41463,"(140, 76, 4, 255)",1
41465,"(140, 76, 14, 255)",1
41466,"(140, 76, 51, 255)",1
41467,"(140, 77, 0, 255)",1


# Display the DataFrame, this requires running in a Jupyter Notebook or an environment that supports HTML.

In [ ]:
styled_df = colorize(color_counts)
styled_df